In [80]:
import pandas as pd

train = pd.read_csv('movies_train.csv')
test = pd.read_csv('movies_test.csv')  # variable name 햇갈리게 지었노
submission = pd.read_csv('submission.csv')

train.head(2)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501


In [81]:
test.head(2)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,300529.00,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,342700.25,4,275,3


In [82]:
submission.head(2)

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0


In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [84]:
train.isnull().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [85]:
# pure dropping
train_dropna = train.dropna()
print(train_dropna.isna().sum())
train_dropna.head(2)

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
dtype: int64


,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083


In [86]:
# NaN -> 0
train_fill = train.fillna(0)
print(train_fill.isna().sum())
train_fill.head(2)

title             0
distributor       0
genre             0
release_time      0
time              0
screening_rat     0
director          0
dir_prev_bfnum    0
dir_prev_num      0
num_staff         0
num_actor         0
box_off_num       0
dtype: int64


,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,0.0,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501


In [87]:
# NaN -> modified mean

train_mean = (train['dir_prev_bfnum'].mean() / 2)  # adding a penalty (이전 관객수가 없는 이유는 신인 감독이니까..)
train_fill_mean = train.fillna(train_mean)
test_fill_mean = test.fillna(train_mean)
# train_fill_mean.head(2)
train_mean

525221.4457098766

In [88]:
# scaling

from sklearn.preprocessing import StandardScaler

features_to_scale = ['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']
scaler = StandardScaler()
features_scaled = scaler.fit_transform(train_fill_mean[features_to_scale])
train_fill_mean_scaled = pd.DataFrame(features_scaled, columns=features_to_scale)
train_fill_mean_scaled.head()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,-0.268953,-0.192527,-0.741416,-0.363216,-0.698066
1,1.611324,0.325860,0.950027,1.425124,-0.289043
2,1.224208,-0.440525,2.641471,1.159290,0.119980
3,0.007558,-0.600902,0.950027,-0.792176,0.938026
4,0.394674,-0.620365,0.104306,0.603454,-0.698066


In [89]:
""" LR with statsmode """

import statsmodels.api as sm

X = train_fill_mean_scaled
y = train['box_off_num']
X = sm.add_constant(X)  # adding a constant to the model for the intercept
model = sm.OLS(y, X).fit()  # order of the variables different from sklearn
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            box_off_num   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                     57.45
Date:                Tue, 06 Feb 2024   Prob (F-statistic):           9.56e-49
Time:                        03:10:35   Log-Likelihood:                -9383.8
No. Observations:                 600   AIC:                         1.878e+04
Df Residuals:                     594   BIC:                         1.881e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           7.082e+05   6.15e+04     11.510      0.000    5.87e+05    8.29e+05
time             2.84e+05   7.91e+04      3.592      0.000    1.29e+05    4.39e+05
dir_prev_bfnum  1.849e+05   6.53e+04      2.831      0.005    5.66e+04    3.13e+05
dir_prev_num    1265.7971   6.96e+04      0.018      0.985   -1.35e+05    1.38e+05
num_staff       7.512e+05   8.52e+04      8.821      0.000    5.84e+05    9.18e+05
num_actor       1.033e+05    6.2e+04      1.667      0.096   -1.84e+04    2.25e+05
==============================================================================
Omnibus:                      514.232   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11749.150
Skew:                           3.785   Prob(JB):                         0.00
Kurtosis:                      23.314   Cond. No.                         2.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
""" LR with sklearn

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = train_fill_mean[['time', 'dir_prev_num', 'num_staff', 'num_actor']]
y = train['box_off_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)
r_squared = model.score(X_test, y_test)
print(f"R^2 on validation data: {r_squared}") """

' LR with sklearn\n\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.model_selection import train_test_split\n\nX = train_fill_mean[[\'time\', \'dir_prev_num\', \'num_staff\', \'num_actor\']]\ny = train[\'box_off_num\']\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)\nmodel = LinearRegression()\nmodel.fit(X_train, y_train)\nr_squared = model.score(X_test, y_test)\nprint(f"R^2 on validation data: {r_squared}") '

In [91]:
# RF regressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = train_fill_mean_scaled
y = train['box_off_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
RF_model = RandomForestRegressor(n_estimators=200, random_state=0)  # You can adjust the parameters
RF_model.fit(X_train, y_train)
y_pred = RF_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.243461676789966

In [92]:
# XGBoost regressor

import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=0)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
r2

0.25143601056762976

결국 나머지 features들을 써야할듯...
심지어는 날짜 feature도 월 또는 분기로 groupby 해서 (왜냠 연말에 개봉되는 영화가 더 유의미하게 관객수가 높을수도 있고 하니까) 추가하고... 등등

In [93]:
train['genre'].value_counts()

드라마       221
다큐멘터리      93
멜로/로맨스     78
코미디        53
공포         42
액션         28
느와르        27
애니메이션      21
미스터리       17
SF         13
뮤지컬         5
서스펜스        2
Name: genre, dtype: int64

In [94]:
train['distributor'].value_counts()

CJ 엔터테인먼트        54
롯데엔터테인먼트         52
(주)NEW           30
(주)마운틴픽쳐스        29
(주)쇼박스           26
                 ..
OAL(올)            1
(주)에이원 엔터테인먼트     1
(주)콘텐츠 윙          1
위더스필름             1
퍼스트런              1
Name: distributor, Length: 169, dtype: int64

In [95]:
train['screening_rat'].value_counts()

청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: screening_rat, dtype: int64

In [103]:
train['director'].value_counts().head(30)

홍상수       7
장률        4
신재호       4
우민호       4
노진수       4
전규환       4
전수일       3
신연식       3
박철수       3
오멸        3
김기덕       3
장진        3
권칠인       3
조성규       3
김현석       3
김조광수      3
김봉은       3
윤여창       3
정성복       3
조조 히데오    3
연상호       2
김동원       2
임순례       2
김태용       2
신수원       2
오영두       2
이상우       2
김성훈       2
김성수       2
송일곤       2
Name: director, dtype: int64

In [ ]:
7 1티어
3~4 까지 2티어
1~2 3티어
0 4티어

이런식으로 범주화해버릴 수 있지 않나...?